<h1 align = "center">Auto-Holidays and Vacation Plan Optimizer</h1>

---

Let the algorithms decide the best optimal days to take holiday breaks from the hustle and bustle of the professional life. The python [**`auto-holidays`**](https://pypi.org/project/auto-holidays) package can be used to find the best possible solution for a single or a group of people considering entitled leave days and weekly off by finding the best possible buckets with the least number of leaves to avail to plan vacations. The extensive advanced uses-cases, like prioritizing certain days of a year, defning and customizing different types of leaves, rolling applicable leaves to a next cycle to find an alternate and better holiday (longer duration) etc. are developed. This *is just the tip of the iceberg*, explore more or send a PR/issue request on [GitHub](https://github.com/ZenithClown/auto-holidays) to collaborate and build something amazing.

In [1]:
import os
import sys

In [2]:
import json

In [3]:
import datetime as dt

In [4]:
sys.path.append(os.path.join(os.path.dirname("__name__"), ".."))
import autoholidays as ah # package in repository root, append to path

## Leaves Models

The module provides the ability to define different types of leaves, and compulsory week-off for a person. The class are `pydantic` base models with validations to prevent abnormal values. Check the documentations for more information about the models. The following types of leaves are available to be defined:

  * **`ah.utils.leaves.CompWeeklyLeave`**: A compulsory weekly-off that is available to a person (typically, Saturday and Sunday). The model also provides advanced configuration - like setting alternate day (example every alternate Saturday) as a holiday, and
  * **`ah.utils.leaves.CustomLeaves`**:A generic model that can be used to define any type of custom holidays - paid/earned/sick/optional/casual/etc. leave with various different types of constraints and controls for more advanced usages - like defining the cycle, maximum balance that can be carry forwarded, etc.

In [5]:
# let's define a custom leave - paid leave which has
#   > a max. balance of 60 in a year,
#   > with an addition of 21 PL at the start of a year, and
#   > only 12 value can be carry forwarded into the next cycle
paid_leave = ah.utils.leaves.CustomLeaves(
    name = "Paid Leave",
    max_balance = 60,

    n_credit_md = [ah.utils.calender.MonthDayConstruct(day = 1, month = 4)],  # only credited once a year
    n_expiry_md = [ah.utils.calender.MonthDayConstruct(day = 31, month = 3)], # any excess expires only once a year

    n_credit_balance = [21],                                                  # for each credit days, no. of balance credited
    carry_forward_balance = 12,                                               # a max. of 12 pl can be carry forwarded in next cycle

    constraints = ah.utils.leaves.CustomLeaveConstraint(
        avail_limit = ah.utils.leaves.LRBoundary(min_value = 4),
        avail_limit_per_cycle = ah.utils.leaves.LRBoundary(max_value = 3),
    )
)

In [6]:
# let's define a custom leave - paid leave which has
#   > a max. balance of 9 in a year,
#   > with an addition of 9 CL at the start of a year, and
#   > and none can be carry forwarded onto the next cycle
casual_leave = ah.utils.leaves.CustomLeaves(
    name = "Casual Leave",
    max_balance = 9,

    n_credit_md = [ah.utils.calender.MonthDayConstruct(day = 1, month = 4)],  # only credited once a year
    n_expiry_md = [ah.utils.calender.MonthDayConstruct(day = 31, month = 3)], # any excess expires only once a year

    n_credit_balance = [9],                                                   # for each credit days, no. of balance credited
    carry_forward_balance = 0,                                                # a max. of 12 pl can be carry forwarded in next cycle

    constraints = ah.utils.leaves.CustomLeaveConstraint(
        avail_limit = ah.utils.leaves.LRBoundary(max_value = 3),
    )
)

### Paid Holidays

A paid holiday is any day of a year (static) which is available to a person (typically, state wise holiday varies between organizations). We can define all the available paid holidays for the current planning cycle - this should be a static value (and can have additional property like `name`, and `priority`). Due to the nature, the dynamic property is not applicable and tracking/storing is out of the scope of this module.

In [7]:
paid_holidays = [
    ah.utils.holidays.HolidayConstruct(
        date = dt.date(2025, 5, 1),
        name = "Maharashtra Day"
    ),
    ah.utils.holidays.HolidayConstruct(
        date = dt.date(2025, 8, 15),
        name = "Independence Day"
    ),
    ah.utils.holidays.HolidayConstruct(
        date = dt.date(2025, 8, 27),
        name = "Ganesh Chaturthi"
    ),
    ah.utils.holidays.HolidayConstruct(
        date = dt.date(2025, 10, 2),
        name = "Gandhi Jayanti"
    ),
    ah.utils.holidays.HolidayConstruct(
        date = dt.date(2025, 10, 20),
        name = "Kali Puja"
    ),
    ah.utils.holidays.HolidayConstruct(
        date = dt.date(2025, 10, 21),
        name = "Diwali Holiday"
    ),
    ah.utils.holidays.HolidayConstruct(
        date = dt.date(2025, 12, 25),
        name = "Christmas Holiday"
    ),
    ah.utils.holidays.HolidayConstruct(
        date = dt.date(2026, 1, 1),
        name = "New Year's Day"
    ),
    ah.utils.holidays.HolidayConstruct(
        date = dt.date(2026, 1, 26),
        name = "Republic Day"
    ),
    ah.utils.holidays.HolidayConstruct(
        date = dt.date(2026, 3, 4),
        name = "Holi Day"
    ),
]

## Leave Planner

In [8]:
planner = ah.core.planner.LeavePlanner(
    period = ah.utils.calender.AnyCycle(
        s = ah.utils.calender.MonthDayConstruct(day = 1, month = 4),
        e = ah.utils.calender.MonthDayConstruct(day = 31, month = 3)
    ),
    persons = [
        ah.utils.person.PersonConstruct(
            name = "John Doe",
            cycle = ah.utils.calender.AnyCycle(
                s = ah.utils.calender.MonthDayConstruct(day = 1, month = 4),
                e = ah.utils.calender.MonthDayConstruct(day = 31, month = 3)
            ),
            custom_leaves = [paid_leave, casual_leave], additional_holidays = paid_holidays
        )
    ]
)

### Paid Holidays

In [16]:
planner.verbose_paid_holidays # no. of paid holidays, ratio/percentage of paid holidays in the planning cycle

{'John Doe': {'total': 114, 'ratio': 0.31319}}

### Long/Extended Weekends Finder

In [17]:
lw = planner.long_weekends(tolerance = 1, min_count = 3) # in-between 1-day tolerance, no. of days >= 3

VERBOSE: Long Weekend(John Doe)
  >> No. of Long Weekends = 54
  >> #LW (#Days >= 3) = 7


In [14]:
print(json.dumps(lw["John Doe"]["subgroups"], default = str, indent = 2)) # details - long weekends plans, with duration and start-end dates

{
  "VACATION #005": {
    "start": "2025-05-01",
    "end": "2025-05-04",
    "duration": 4,
    "values": null
  },
  "VACATION #020": {
    "start": "2025-08-15",
    "end": "2025-08-17",
    "duration": 3,
    "values": null
  },
  "VACATION #028": {
    "start": "2025-10-02",
    "end": "2025-10-05",
    "duration": 4,
    "values": null
  },
  "VACATION #030": {
    "start": "2025-10-18",
    "end": "2025-10-21",
    "duration": 4,
    "values": null
  },
  "VACATION #040": {
    "start": "2025-12-25",
    "end": "2025-12-28",
    "duration": 4,
    "values": null
  },
  "VACATION #041": {
    "start": "2026-01-01",
    "end": "2026-01-04",
    "duration": 4,
    "values": null
  },
  "VACATION #044": {
    "start": "2026-01-24",
    "end": "2026-01-26",
    "duration": 3,
    "values": null
  }
}
